## CIS5560: PySpark Gradient Boosted Tree Regression in Databricks

### by Team 4 (Uche, Raymond, Tofunmi and Sweta) edited on 05/15/2020
Tested in Runtime 6.5 (Spark 2.4.5/2.4.0 Scala 2.11) of Databricks CE

## Steps to download dataset and do some data engineering (Cleaning up dataset) before importing into databricks


all dataset engineering were done in Jupyter Notebook before importing into databricks

dataset link: https://www.kaggle.com/darshank2019/review#yelp_academic_dataset_review.csv

download dataset and using a Jupyter Notebook(we used google colab), we accessed the dataset with total rows = 6685900

we took a slice of the full dataset of the first 1500000 rows and used that as our full dataset.

we removed the inverted commas and the letter "b" present in all rows (data cleaning)

we converted the alphanumeric values in the user_id, review_id, & business_id to numeric values

we tried to drop rows wit missing values and counted the total number of rows again and it was still 1500000.

we created a subset of our cleaned dataset named df_ml_csv with 120000 rows which we used for both Azure ML & Databricks

NOTE: the .py & .ipynb files containing all codes used for data engineering and analysis is included in the total submission package and is availble in our github link

## For this project, we further normalised the user_id, review_id, and business_id columns of our df_ml dataset(subset with 120000 rows) 

Normalised dataset is named scaled_subset 

scaled_subset is imported into databricks and used for Gradient Boost classification model

NOTE: Codes used for normalisation of the above listed columns are contained in thedata engineering and analysis .py & ipynb files uploaded to the github link

Import the scaled_subset.csv dataset

##Prepare the Data
First, import the libraries you will need and prepare the training and test data:

In [6]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
from pyspark.ml.classification import GBTClassifier

from pyspark.sql import functions as F
import pyspark.sql.functions as func

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml import Pipeline
from pyspark.sql.types import DoubleType
from pyspark.ml.linalg import Vectors, SparseVector
import re


## Create a DataFrame Schema, 
that should be a Table schema

In [8]:
# DataFrame Schema, that should be a Table schema by Team 4 
df_mlSchema = StructType([
  StructField("user_id", IntegerType(), False),
  StructField("text", StringType(), False),
  StructField("date", TimestampType(), False),
  StructField("review_id", IntegerType(), False),
  StructField("business_id", IntegerType(), False),
  StructField("funny", IntegerType(), False),
  StructField("cool", IntegerType(), False),
  StructField("useful", IntegerType(), False),
  StructField("stars", IntegerType(), False),
])

In [9]:
%fs ls /FileStore/tables/df_ml.csv

path,name,size
dbfs:/FileStore/tables/df_ml.csv,df_ml.csv,77730282


In [10]:
IS_SPARK_SUBMIT_CLI = True
if IS_SPARK_SUBMIT_CLI:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)

##Load Dataset 

ensure command line above: IS_SPARK_SUBMIT_CLI = False. Also remember to set it to 'True' before exporting

Read csv file from DBFS (Databricks File Systems)

## follow the direction to read your table after upload it to Data at the left frame
NOTE: See above for the data type - 

After df_ml_csv file is added to the data of the left frame, create a table using the UI, especially, "Upload File"
tick header and infer schema before creating table

In [13]:
if IS_SPARK_SUBMIT_CLI:
   df_ml = spark.read.csv('df_norm.csv', inferSchema=True, header=True)
else:
    df_ml = spark.sql("SELECT * FROM scaled_subset_csv")

In [14]:
df_ml.show(5)

+---+------------------+--------------------+-------------------+--------------------+--------------------+-----+----+------+-----+
_c0| user_id| text| date| review_id| business_id|funny|cool|useful|stars|
+---+------------------+--------------------+-------------------+--------------------+--------------------+-----+----+------+-----+
 0|0.4875106473594548|Total bill for th...|2013-05-07 04:34:36| 0.06781723181026508| 0.7374873010497799| 1| 0| 6| 1.0|
 1|0.7909071550255536|I *adore* Travis ...|2017-01-14 21:30:33| 0.10313419278493988| 0.04551303758889265| 0| 0| 0| 5.0|
 2|0.5396614139693355|I have to say tha...|2016-11-09 20:09:03|0.024133534446120382| 0.3907213003725025| 0| 0| 3| 5.0|
 3| 0.530291737649063|Went in for a lun...|2018-01-09 20:56:38| 0.4730956091300761|0.012258719945817812| 0| 0| 0| 5.0|
 4| 0.554557069846678|Today was my seco...|2018-01-30 23:07:38| 0.12229268577238143| 0.5214358279715544| 0| 0| 7| 1.0|
+---+------------------+--------------------+-------------------+--------------------+--------------------+-----+----+------+-----+
only showing top 5 rows

##Create a New Dataframe with columns "user_id", "review_id", "business_id" and "stars"(label)
The label is the stars (stars > 2 = 1 (positive review) else: 0 (negative review)

These are the columns we used in building of Gradient Boost Classifier Model

In [16]:
data = df_ml.select("user_id", "review_id", "business_id", ((col("stars") > 2).cast("Double").alias("label")))

data.show(5)

+------------------+--------------------+--------------------+-----+
 user_id| review_id| business_id|label|
+------------------+--------------------+--------------------+-----+
0.4875106473594548| 0.06781723181026508| 0.7374873010497799| 0.0|
0.7909071550255536| 0.10313419278493988| 0.04551303758889265| 1.0|
0.5396614139693355|0.024133534446120382| 0.3907213003725025| 1.0|
 0.530291737649063| 0.4730956091300761|0.012258719945817812| 1.0|
 0.554557069846678| 0.12229268577238143| 0.5214358279715544| 0.0|
+------------------+--------------------+--------------------+-----+
only showing top 5 rows

In [17]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")
train_rows = train.count()
test_rows = test.count()
print ("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 83694 Testing Rows: 36306

### Build the Recommender
 user_id, review_id and business_id are columns we used to build the Gradient Boost Classifier Model.

#### Latent Features
We can use the features to produce some sort of algorithm (**GBTRegression**) to intelligently calculate stars(ratings) 

The GBT class is an estimator, so you can use its **fit** method to traing a model, or you can include it in a pipeline. Rather than specifying a feature vector and as label, the GBT algorithm requries user_id, review_id and business_id columns are Normalized
NOTE: all columns are normalized in python jupyter notebook before dataframe was imported

In [19]:
gbtassembler = VectorAssembler(inputCols=["user_id", "review_id", "business_id"], outputCol="features")

In [20]:
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10) 

In [21]:
gbtp = Pipeline(stages=[gbtassembler, gbt])

#### Add paramGrid and Validation

In [23]:
paramGrid = (ParamGridBuilder()
             .addGrid(gbt.maxDepth,[2,3,4])
             .addGrid(gbt.maxBins, [49, 52, 55])
             .addGrid(gbt.minInfoGain,[0.0, 0.1, 0.2, 0.3])
             .addGrid(gbt.stepSize,[0.05, 0.1, 0.2, 0.4])
         
             .build())


### To build a general model, _TrainValidationSplit_ is used by us as it is much faster than _CrossValidator_
CrossValidator takes a very long time to run.

In [25]:
gbt_tvs = TrainValidationSplit(estimator=gbtp, evaluator=MulticlassClassificationEvaluator(), estimatorParamMaps=paramGrid, trainRatio=0.8)

gbtModel = gbt_tvs.fit(train)


### Test the Recommender
Now that we've trained the recommender, lets see how accurately it predicts known stars in the test set.

In [27]:
prediction = gbtModel.transform(test)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show(10)

+--------------------+----------+---------+
 features|prediction|trueLabel|
+--------------------+----------+---------+
[1.27768313458262...| 1.0| 0.0|
[5.85604770017035...| 1.0| 1.0|
[8.62436115843270...| 1.0| 0.0|
[0.00102214650766...| 1.0| 0.0|
[0.00121379897785...| 1.0| 0.0|
[0.00122444633730...| 1.0| 1.0|
[0.00143739352640...| 1.0| 1.0|
[0.00158645655877...| 1.0| 1.0|
[0.00166098807495...| 1.0| 1.0|
[0.00190587734241...| 1.0| 0.0|
+--------------------+----------+---------+
only showing top 10 rows

##TP, FP, TN, and FN all calculated
Precision and recall also calculated

In [29]:
tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())
metrics = spark.createDataFrame([
      ("TP", tp),
      ("FP", fp),
      ("TN", tn),
      ("FN", fn),
      ("Precision", tp / (tp + fp)),
      ("Recall", tp / (tp + fn))],["metric", "value"])
metrics.show()

+---------+------------------+
 metric| value|
+---------+------------------+
 TP| 28083.0|
 FP| 8204.0|
 TN| 7.0|
 FN| 12.0|
Precision|0.7739135227491939|
 Recall| 0.999572877736252|
+---------+------------------+

## AUC is calculated

In [31]:
gbt_evaluator =  MulticlassClassificationEvaluator(labelCol="trueLabel", predictionCol="prediction")
gbt_auc = gbt_evaluator.evaluate(prediction)

print("AUC for Gradient Boost Classifier = ", gbt_auc)

AUC for Gradient Boost Classifier = 0.675471596680073

## AUC for Gradient Boost Classifier =  0.675471596680073